In [1]:
pip install galois

Note: you may need to restart the kernel to use updated packages.


In [2]:
import galois
import numpy as np
import threading

np.set_printoptions(threshold=np.inf)

In [3]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(127)

In [4]:
#definisco una matrice k x (n-k) dove n-k = 4
k = 126 #le righe
n_k = 126 #le colonne

#N.B: Less usa q = dimensione campo: 127; k = 126; n = 252 

In [5]:
#provo co due matrici diagonali NON funziona
valori_diagonale = np.arange(1,127)
A = np.diag(valori_diagonale)

perm = np.random.permutation(126) #permutazione degli indici casuale
A_primo = A[perm, :]
A_primo = A_primo[:, perm]

A = field(A)
A_primo = field(A_primo)

In [5]:
A = field.Random((k, n_k))
#print(A)

In [7]:
id_mat = np.eye(126, dtype=int)

cols_to_permute = np.random.choice(id_mat.shape[1], size=50, replace=False)
permuted_cols = np.random.permutation(cols_to_permute)
matrice_bin1 = id_mat.copy()
matrice_bin1[:, cols_to_permute] = id_mat[:, permuted_cols]
matrice_bin1 = field(matrice_bin1)

cols_to_permute = np.random.choice(id_mat.shape[1], size=50, replace=False)
permuted_cols = np.random.permutation(cols_to_permute)
matrice_bin2 = id_mat.copy()
matrice_bin2[:, cols_to_permute] = id_mat[:, permuted_cols]
matrice_bin2 = field(matrice_bin2)

print(matrice_bin1)
print(matrice_bin2)

[[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [8]:
#np.dot prodotto di due  array
A_primo_intermedio = field(np.dot(matrice_bin1, A))
#print(A_primo_intermedio)

In [9]:
A_primo = matrice_bin1 @ A @matrice_bin2
A_primo = field(A_primo)
#print(A_primo)

In [10]:
#qui in accordo con slide 18 articolo devo ordinare la matrice A e A_primo per righe e colonne, ordinamento per riga 
#è dato dalla somma delle righe in ordine decrescente, mentre per colonna in ordine numerico del primo elemento e in caso di 
#parità del secondo e così via

#lista per conservare somme delle righe, inizializzata con gli zeri
somme_rig = [field(0)] * A_primo.shape[0]

#lista per memorizzare i primi elementi della colonna
primi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i secondi elemnti della colonna
secondi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i terzi elementi della colonna
terzi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i quarti elemnti della colonna
quarti_colonne = [0] * A_primo.shape[1]

#QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
#PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 


#funzione per somma elementi nella colonna
def somma_riga(matrice, indice_riga):
    riga = matrice[indice_riga, :]
    somma = field(np.sum([riga])%127)

    #print("Riga : ", riga) #questi vanno eliminati poi
    #print("Tipo riga: ", type(riga)) #questo è per far vedere che pur essendo di tipo GF il np.sum non funziona se non metto %8
    #print("Somma: ", somma) #questi vanno eliminati poi
    #print("Tipo somma: ",type(somma)) #questo è per far vedere che pur essendo di tipo GF il np.sum non funziona se non metto %8
    somme_rig[indice_riga] = somma
        #print(somme_rig)

#vado ad estrarre il primo ed il secondo elemento di ogni colonna
def estrai_da_colonna(matrice, indice_colonna):
    colonna  = matrice[:, indice_colonna]
    primi_colonne[indice_colonna] = colonna[0]
    #print(primi_colonne)
    secondi_colonne[indice_colonna] = colonna[1]
    #print(secondi_colonne)
    terzi_colonne[indice_colonna] = colonna[2]
    quarti_colonne[indice_colonna] = colonna[3]



#eseguo in parallelo somma_riga
def esegui_somma_righe_parallelo(matrice):

    for indice_riga in range(matrice.shape[0]):
        somma_riga(matrice, indice_riga)

#eseguo in parallelo estrazione
def esegui_estrazione_parallelo(matrice):

    for indice_colonna in range(matrice.shape[1]):
        estrai_da_colonna(matrice, indice_colonna)

#Funzione per applicare una permutazione ciclica a un insieme di colonne (ciclicità aiuta a mantenere una certa struttura)
def permutazione_ciclica(matrice, blocco_dim):
    matrice_permutata = matrice.copy()
    
    # Eseguiamo una permutazione ciclica su tutte le colonne a gruppi di 'blocco_dim'
    for i in range(0, matrice.shape[1], blocco_dim):
        fine_blocco = min(i + blocco_dim, matrice.shape[1])
        # Cicla il blocco selezionato
        matrice_permutata[:, i:fine_blocco] = np.roll(matrice[:, i:fine_blocco], shift=1, axis=1)
    
    return field(matrice_permutata)


In [11]:
#fino a size 3 provato e funziona
while True:
    id_mat = np.eye(126, dtype=int)

    cols_to_permute = np.random.choice(id_mat.shape[1], size=3, replace=False)
    permuted_cols = np.random.permutation(cols_to_permute)
    matrice_bin1 = id_mat.copy()
    matrice_bin1[:, cols_to_permute] = id_mat[:, permuted_cols]
    matrice_bin1 = field(matrice_bin1)

    cols_to_permute = np.random.choice(id_mat.shape[1], size=3, replace=False)
    permuted_cols = np.random.permutation(cols_to_permute)
    matrice_bin2 = id_mat.copy()
    matrice_bin2[:, cols_to_permute] = id_mat[:, permuted_cols]
    matrice_bin2 = field(matrice_bin2)

    #print(matrice_bin1)
    #print(matrice_bin2)
    A_primo = matrice_bin1 @ A @ matrice_bin2
    A_primo = field(A_primo)
    #print(A_primo)"""
    somme_rig = [field(0)] * A_primo.shape[0]

    #lista per memorizzare i primi elementi della colonna
    primi_colonne = [0] * A_primo.shape[1]
    #lista per memorizzare i secondi elemnti della colonna
    secondi_colonne = [0] * A_primo.shape[1]
    #lista per memorizzare i terzi elementi della colonna
    terzi_colonne = [0] * A_primo.shape[1]
    #lista per memorizzare i quarti elemnti della colonna
    quarti_colonne = [0] * A_primo.shape[1]

    #QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
    #PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 

    colonne_finali = []
    esegui_somma_righe_parallelo(A_primo)
    #ordino in maniera crescente per valore gli indici
    indici_ordinati = np.argsort(somme_rig)
        #print(indici_ordinati)

        #applico gli swap
    A_canonica_righe = A_primo[indici_ordinati, :]
    #ora si passa alle colonne
    esegui_estrazione_parallelo(A_canonica_righe)

    #una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
    #print("Primi colonne: ", primi_colonne)
    indici_ordinati_col = np.argsort(primi_colonne)
    #print("Indici ordinati col: ", indici_ordinati_col)


    for idx in indici_ordinati_col:
        #print("Index: ", idx)
        #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
        colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
        #print("Colonne finali: ", colonne_finali)

    #vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
    colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

    #qui prendiamo tutte le righe (prima dei :) e alle colonne per ogni colonna di colonne_finali accediamo al 3 elemento (l'indice)
    A_CF = A_canonica_righe[:, [col[4] for col in colonne_finali]]

    #print(A_CF)

    #devo fare lo stesso per la matrice A per dimostrare che sono CF

    colonne_finali = []

    #creo un thread per ogni colonna
    esegui_somma_righe_parallelo(A)

    #print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)

    #ordino in maniera crescente i valori
    indici_ordinati = np.argsort(somme_rig)
    #print(indici_ordinati)

    #applico gli swap
    A_canonica_righe = A[indici_ordinati, :]

    #print(A_canonica_righe)

    #ora si passa alle colonne
    esegui_estrazione_parallelo(A_canonica_righe)

    #una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
    #print("Primi colonne: ", primi_colonne)
    indici_ordinati_col = np.argsort(primi_colonne)
    #print("Indici ordinati col: ", indici_ordinati_col)


    for idx in indici_ordinati_col:
        #print("Index: ", idx)
        #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
        colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
        #print("Colonne finali: ", colonne_finali)

    #vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
    colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

    A_CF1 = A_canonica_righe[:, [col[4] for col in colonne_finali]]
    #print(A_CF1)

    zero = np.subtract(A_CF1,A_CF)
    zero = np.count_nonzero(zero)
    print("Le matrici differiscono per ", zero, " elementi su 15876 (126*126)")
    if zero == 0:
        break


Le matrici differiscono per  1252  elementi su 15876 (126*126)
Le matrici differiscono per  0  elementi su 15876 (126*126)


In [12]:
print("Questa è A:", A)
print("\nQuesta è A': ", A_primo)
print("\nQuesta è la forma canonica: ", A_CF)

Questa è A: [[102  36  73 109  86  13  83 101 115 114 111  55  17   0 113 115  27 114
   71  91 112  46  37   4  30  28  54 101   6 110  98 105 101 122  68  78
   99  18  41  87  64  29  91 109   7  55  14  56  80  65  20 105   5   8
   25  81  65  28  44   9  31 102  24  46   3  25  77  23 107 108  26  69
  107  61  25 103  13  23  76  67  85 103  29  60  41  43  99  22  51  26
   31  37  41  13  51  57  45  47  97  54 113  36  93  62 109  85  50  48
   39  76  59  30  29  58  68 112  65 109  48  96  29  46  14  65  44  62]
 [ 11  65  90  44 115  15  12  77 119  90  53  41  56  41  11 105  79  43
   29  30  48 101 105 119  44 103  27   2  25  66  85  49   0  84  56  65
   41 126  49  79  64  74 117  76  27  77  16  10  18  89  22  18  93  59
  105  98  70  31  12  12  50  13  12 101  17  42  41  36 108   5 121 118
  109 101  11  13  17  65 100  34  82  55   4  80  26  89  50  95 119  64
   57  54  50  18 104 103  52 105 105  76 114  59  69  77  36   1 112  69
  117  20 105  76   9  80

In [13]:
import time

In [23]:
#Voglio vedere il tempo di esecuzione avendo già le matrici

#lista per memorizzare i primi elementi della colonna
primi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i secondi elemnti della colonna
secondi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i terzi elementi della colonna
terzi_colonne = [0] * A_primo.shape[1]
#lista per memorizzare i quarti elemnti della colonna
quarti_colonne = [0] * A_primo.shape[1]

#QUI SI POTREBBE RAGGIONARE SU QUANTI ELEMENTI SALVARE DI OGNI COLONNA POICHÈ LA POSSIBILITÀ CHE QUESTI SIANO UGUALI 
#PER PIÙ DI TOT RIGHE DIMINUISCE (1/PROB_CONFLITTO_RIGA1 * 1/PROB_CONFLITTO_RIGA2 ....) È DA VEDERE COME VENGONO CREATE 

colonne_finali = []
start_time = time.perf_counter()

esegui_somma_righe_parallelo(A_primo)
#ordino in maniera crescente per valore gli indici
indici_ordinati = np.argsort(somme_rig)
    #print(indici_ordinati)

    #applico gli swap
A_canonica_righe = A_primo[indici_ordinati, :]
#ora si passa alle colonne
esegui_estrazione_parallelo(A_canonica_righe)

#una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
#print("Primi colonne: ", primi_colonne)
indici_ordinati_col = np.argsort(primi_colonne)
#print("Indici ordinati col: ", indici_ordinati_col)


for idx in indici_ordinati_col:
    #print("Index: ", idx)
    #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
    #print("Colonne finali: ", colonne_finali)

#vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

#qui prendiamo tutte le righe (prima dei :) e alle colonne per ogni colonna di colonne_finali accediamo al 3 elemento (l'indice)
A_CF = A_canonica_righe[:, [col[4] for col in colonne_finali]]

#print(A_CF)

#devo fare lo stesso per la matrice A per dimostrare che sono CF

colonne_finali = []

#creo un thread per ogni colonna
esegui_somma_righe_parallelo(A)

#print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)

#ordino in maniera crescente i valori
indici_ordinati = np.argsort(somme_rig)
#print(indici_ordinati)

#applico gli swap
A_canonica_righe = A[indici_ordinati, :]

#print(A_canonica_righe)

#ora si passa alle colonne
esegui_estrazione_parallelo(A_canonica_righe)

#una volta fatta la funzione di estrazione dei valori ordino gli indici in maniera crescente per valore
#print("Primi colonne: ", primi_colonne)
indici_ordinati_col = np.argsort(primi_colonne)
#print("Indici ordinati col: ", indici_ordinati_col)

for idx in indici_ordinati_col:
    #print("Index: ", idx)
    #qui tengo il primo e il secondo della colonna e gli associo l'indice di riga
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], terzi_colonne[idx], quarti_colonne[idx], idx))
    #print("Colonne finali: ", colonne_finali)

#vado a fare ordinamento per il primo elemento e in caso per il secondo, qui nel caso devo inserire il terzo quarto...
colonne_finali.sort(key=lambda x: (x[0], x[1], x[2], x[3]))

A_CF1 = A_canonica_righe[:, [col[4] for col in colonne_finali]]
#print(A_CF1)
end_timer = time.perf_counter()
durata = (end_timer - start_time)*1000
print("Durata esecuzione: ", durata, " ms")

zero = np.subtract(A_CF1,A_CF)
zero = np.count_nonzero(zero)
print("Le matrici differiscono per ", zero, " elementi su 15876 (126*126)")

Durata esecuzione:  65.67767900014587  ms
Le matrici differiscono per  0  elementi su 15876 (126*126)
